# <center>**Import necessary libraries**</center>


In [1801]:
import pathlib
import pandas as pd
import math
import design_functions.rebar_information as rebar_func

## <center>Import and read the desired beam flexural and shear reinforcement excel file from ETABS</center>

In [1802]:
excel_file = pathlib.Path('excel_files\example_analysis.xlsx')
initial_df = pd.read_excel(excel_file, sheet_name=None)
pd.set_option('display.max_rows', None)

## <center>Because the excel sheet has two sheets, flexural and shear, they need to be read individually</center>

## <center>Delete unnecessary rows and columns for simplicity</center>

V1 of respective df's = first two rows are deleted<br>
V2 of respective df's = unique name, negative moment, negative combo, positive moment, and positive combo columns are removed.<br>
V3 of respective df's = each unique beam is consolidated to extract information such as section size, unqiue name.<br>
V4 of respective df's = ETABs section column is removed.<br>
For shear, identifying columns are removed as they are not needed. Additionally, unique beams DF doesn't need to be made as they can be taken from flexural DF. Therefore V2 may be primarily used

In [1803]:
v1_flexural_df = initial_df['Sheet1'].drop([0,1])
v1_shear_df = initial_df['Sheet2'].drop([0,1])

In [1804]:
v2_flexural_df = v1_flexural_df.drop(['Unnamed: 2', 'Unnamed: 5', 'Unnamed: 6', 'Unnamed: 8', 'Unnamed: 9'], axis=1)
v2_shear_df = v1_shear_df.drop(['Unnamed: 1', 'Unnamed: 2', 'Unnamed: 3', 'Unnamed: 4', 'Unnamed: 5', 'Unnamed: 6', 'Unnamed: 8', 'Unnamed: 9', 'Unnamed: 11', 'Unnamed: 12'], axis=1)

In [1805]:
v3_flexural_df = v2_flexural_df.iloc[::3]

## <center>Create width and depth columns to utilise in calculations</center>

In [1806]:
v3_flexural_df.insert(3, 'Width (mm)', None)
v3_flexural_df.insert(4, 'Depth (mm)', None)

In [1807]:
#this function cleans the cell of unnamed: 3 column to provide the width of each respective beam.
def clean_width_dimensions(width):
    width_list = list(width) # turn string into list of individual indexes
    width_list = [el.lower() for el in width_list] #use list comprehension to turn list into lower case values
    excluded_values = ['p', 't', 'b', '-', '_', 'c', '/'] #create list of excluded indices
    v1_width_list = [ex for ex in width_list if ex not in excluded_values] #use list comprehension to return list excluding indices
    index_list = v1_width_list.index('x') #index the list to x to retrieve required width
    v2_width_list = v1_width_list[:index_list] #slice the width list to the index x
    true_width = ''.join(v2_width_list) #join the list into a string
    return int(true_width) #turn string into int so it can be used in other functions

In [1808]:
v3_flexural_df.loc[:, 'Width (mm)'] = v3_flexural_df['Unnamed: 3'].apply(clean_width_dimensions)

In [1809]:
#this function cleans the cell of unnamed: 3 column to provide the depth of each respective beam.
#this function follows the same steps as clean_width_dimensions function
def clean_depth_dimensions(depth):
    depth_list = list(depth)
    depth_list = [el.lower() for el in depth_list]
    excluded_values = ['p','t', 'b', '-', '_', 'c', '/']
    v1_depth_list = [ex for ex in depth_list if ex not in excluded_values]
    index_list = v1_depth_list.index('x')
    v2_depth_list = v1_depth_list[1+index_list:-4]
    true_depth = ''.join(v2_depth_list)
    return int(true_depth)

In [1810]:
v3_flexural_df.loc[:, 'Depth (mm)'] = v3_flexural_df['Unnamed: 3'].apply(clean_depth_dimensions)

## <center>With the width and depth defined, remove the ETABS section, location, and rebar area columns</center>

In [1811]:
v4_flexural_df = v3_flexural_df.drop(['Unnamed: 3', 'Unnamed: 4', 'Unnamed: 7', 'Unnamed: 10'], axis=1)

In [1812]:
v4_flexural_df = v4_flexural_df.rename(columns={'Unnamed: 1': 'ETABS beam ID'})
v2_shear_df = v2_shear_df.rename(columns={'Unnamed: 7': 'Shear rebar area (mm2/m)', 'Unnamed: 10': 'Shear tension rebar area (mm2/m)', 'Unnamed: 13': 'Longutudinal tension rebar area (mm2)'})

## <center>Create Dataframe to store beam reinforcement schedule</center>

In [1813]:
# Create dataframe which reflects column headings
columns = pd.MultiIndex.from_tuples([
    ('Storey', ''),
    ('Etabs ID', ''),
    ('Dimensions', 'Width (mm)'),
    ('Dimensions', 'Depth (mm)'),
    ('Bottom Reinforcement', 'Left (BL)'),
    ('Bottom Reinforcement', 'Middle (B)'),
    ('Bottom Reinforcement', 'Right (BR)'),
    ('Top Reinforcement', 'Left (TL)'),
    ('Top Reinforcement', 'Middle (T)'),
    ('Top Reinforcement', 'Right (TR)'),
    ('Shear links', 'Left (H)'),
    ('Shear links', 'Middle (J)'),
    ('Shear links', 'Right (K)'),
    ('Side Face Reinforcement', '')
])
beam_schedule_df = pd.DataFrame(columns=columns)

## <center>For each index of the v4 respective df, populate the main beam_schedule_df with the necessary information</center>

In [1814]:
beam_schedule_df['Storey'] = v4_flexural_df['TABLE:  Concrete Beam Flexure Envelope - ACI 318-19']
beam_schedule_df['Etabs ID'] = v4_flexural_df['ETABS beam ID']
beam_schedule_df['Dimensions', 'Width (mm)'] = v4_flexural_df['Width (mm)']
beam_schedule_df['Dimensions', 'Depth (mm)'] = v4_flexural_df['Depth (mm)']
beam_schedule_df.reset_index(drop=True, inplace=True)
beam_schedule_df

Storey Etabs ID Dimensions            Bottom Reinforcement              
                   Width (mm) Depth (mm)            Left (BL) Middle (B)   
0      P4     B213        300        500                  NaN        NaN  \
1      P4     B215        300        500                  NaN        NaN   
2      P4     B232        600        600                  NaN        NaN   
3      P4     B235        300        500                  NaN        NaN   
4      P4     B237        300        500                  NaN        NaN   
5      P4     B241        300        600                  NaN        NaN   
6      P4     B257        400        600                  NaN        NaN   
7      P4     B260        600        600                  NaN        NaN   
8      P4     B332        300        600                  NaN        NaN   
9      P4     B333        600        600                  NaN        NaN   
10     P4     B342        300        600                  NaN        NaN   
11     P4     B381        300        600                  NaN        NaN   
12     P4     B400        600        600                  NaN        NaN   
13     P4     B403        600        600                  NaN        NaN   
14     P4     B412        600        600                  NaN        NaN   
15     P4     B483        500        800                  NaN        NaN   
16     P4     B895        600        600                  NaN        NaN   
17     P4     B896        600        600                  NaN        NaN   
18     P4     B897        600        600                  NaN        NaN   
19     P4     B899        500        800                  NaN        NaN   
20     P4     B900        500        600                  NaN        NaN   
21     P4     B901        500        800                  NaN        NaN   
22     P4      B78        400        800                  NaN        NaN   
23     P4     B617        400        800                  NaN        NaN   
24     P4     B391        400        600                  NaN        NaN   
25     P4      B12        600        600                  NaN        NaN   
26     P4     B713        400        600                  NaN        NaN   
27     P4     B392        400       1000                  NaN        NaN   
28     P4      B35        500        800                  NaN        NaN   
29     P4     B328        300        600                  NaN        NaN   
30     P4     B590        300        600                  NaN        NaN   
31     P4     B653        200        500                  NaN        NaN   
32     P4     B712        200        500                  NaN        NaN   

              Top Reinforcement                       Shear links              
   Right (BR)         Left (TL) Middle (T) Right (TR)    Left (H) Middle (J)   
0         NaN               NaN        NaN        NaN         NaN        NaN  \
1         NaN               NaN        NaN        NaN         NaN        NaN   
2         NaN               NaN        NaN        NaN         NaN        NaN   
3         NaN               NaN        NaN        NaN         NaN        NaN   
4         NaN               NaN        NaN        NaN         NaN        NaN   
5         NaN               NaN        NaN        NaN         NaN        NaN   
6         NaN               NaN        NaN        NaN         NaN        NaN   
7         NaN               NaN        NaN        NaN         NaN        NaN   
8         NaN               NaN        NaN        NaN         NaN        NaN   
9         NaN               NaN        NaN        NaN         NaN        NaN   
10        NaN               NaN        NaN        NaN         NaN        NaN   
11        NaN               NaN        NaN        NaN         NaN        NaN   
12        NaN               NaN        NaN        NaN         NaN        NaN   
13        NaN               NaN        NaN        NaN         NaN        NaN   
14        NaN               NaN        NaN        NaN         

## <center>Create a dictionary of each unique beam from v3 df, and associate the relevant top / bottom rebar requirement with each </center>

In [1815]:
bottom_beam_rebar = {}
values_per_key = 3
values_counter = 0
for i, row in v3_flexural_df.iterrows():
    key = row['Unnamed: 1']
    values = v2_flexural_df.iloc[values_counter:values_counter+values_per_key]['Unnamed: 7'].tolist()
    bottom_beam_rebar[key] = values
    values_counter += values_per_key

In [1816]:
top_beam_rebar = {}
values_per_key = 3
values_counter = 0
for i, row in v3_flexural_df.iterrows():
    key = row['Unnamed: 1']
    values = v2_flexural_df.iloc[values_counter:values_counter+values_per_key]['Unnamed: 10'].tolist()
    top_beam_rebar[key] = values
    values_counter += values_per_key

## <center>Create Dataframe to do flexural calculations </center>

In [1817]:
columns = pd.MultiIndex.from_tuples([
    ('Etabs ID', ''),
    ('Bottom Reinforcement', 'Left (BL)'),
    ('Bottom Reinforcement', 'Middle (B)'),
    ('Bottom Reinforcement', 'Right (BR)'),
    ('Top Reinforcement', 'Left (TL)'),
    ('Top Reinforcement', 'Middle (T)'),
    ('Top Reinforcement', 'Right (TR)'),
])
flexural_calc_df = pd.DataFrame(columns=columns)
flexural_calc_df

Empty DataFrame
Columns: [(Etabs ID, ), (Bottom Reinforcement, Left (BL)), (Bottom Reinforcement, Middle (B)), (Bottom Reinforcement, Right (BR)), (Top Reinforcement, Left (TL)), (Top Reinforcement, Middle (T)), (Top Reinforcement, Right (TR))]
Index: []

In [1818]:
#assign etabs beam id to flexural calc df
flexural_calc_df['Etabs ID'] = v3_flexural_df['Unnamed: 1'] 

#turn dictionaries into dataframes
top_beam_rebar = pd.DataFrame(top_beam_rebar)
true_top_beam_rebar = top_beam_rebar.T
bottom_beam_rebar = pd.DataFrame(bottom_beam_rebar)
true_bottom_beam_rebar = bottom_beam_rebar.T
#combine datafrmes, with bottom rebar being first 3 columns
v1_flexural_calc_df = pd.concat([bottom_beam_rebar, top_beam_rebar], axis=0)
v2_flexural_calc_df = v1_flexural_calc_df.T
# v2_flexural_calc_df = v2_flexural_calc_df.rename(columns={'213': 'Bottom reinforcement (mm)'})
# v2_flexural_calc_df

,0,1,2,0,1,2
B213,3,229,302,28,90,124
B215,1683,1356,1357,2038,1651,1821
B232,1563,690,1388,1376,998,1083
B235,14,14,14,85,122,85
B237,99,441,203,45,441,239
B241,481,389,341,362,301,284
B257,722,460,553,553,489,722
B260,1638,863,1712,1083,1300,908
B332,541,336,482,541,366,371
B333,1708,871,1771,948,1171,970
